# Prerequisite

## Load "users_001.csv" into dataframe

In [0]:
df = spark.read.csv(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv",
    header=True,
    inferSchema=True,
)
df.count()

## Transaction 01- Write in delta format

In [0]:
df.write.format("delta").save(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta"
)

# Overwrite

In [0]:
from pyspark.sql.functions import col
df.filter(col("country")=="India").write.mode("overwrite").save(
     "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta",
    mode="overwrite",
)

In [0]:
spark.read.load( "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").count()

In [0]:
spark.read.option("versionAsOf",0).load( "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").count()